In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130560

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,113602.2330
2018-02-28,102083.6598
2018-03-31,100297.9334
2018-04-30,80279.0991
2018-05-31,59155.3669
2018-06-30,83013.6574
2018-07-31,64092.6904
2018-08-31,73820.6131
2018-09-30,75902.7746


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    113602.2330
 2018-02-28    102083.6598
 2018-03-31    100297.9334
 2018-04-30     80279.0991
 2018-05-31     59155.3669
 2018-06-30     83013.6574
 2018-07-31     64092.6904
 2018-08-31     73820.6131
 2018-09-30     75902.7746
 2018-10-31     88094.9330
 2018-11-30     83595.0460
 2018-12-31    159421.0605
 2019-01-31    266635.2927
 2019-02-28    168004.8165
 2019-03-31    252206.1045
 2019-04-30    223880.1500
 2019-05-31    144384.6959
 2019-06-30    125753.5730
 2019-07-31    116456.4991
 2019-08-31    149114.8134
 2019-09-30    110333.1355
 2019-10-31    155505.9176
 2019-11-30    144700.9448
 2019-12-31    163322.7477
 2020-01-31    150982.0935
 2020-02-29    170110.7494
 2020-03-31    245837.5296
 2020-04-30    209228.3028
 2020-05-31    224606.1995
 2020-06-30    132594.5736
 2020-07-31    142429.4870
 2020-08-31    125193.9284
 2020-09-30    160739.5434
 2020-10-31    179975.5583
 2020-11-30    210528.9119
 2020-12-31    245570.2409

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.825328
p-value : 0.054746
Critical Values :
	1%: -3.615509
	5%: -2.941262
	10%: -2.609200


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -8.294187
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=997.722, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=996.722, Time=0.08 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=999.000, Time=0.05 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=999.002, Time=0.11 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=997.913, Time=0.13 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.46 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=996.280, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=998.360, Time=0.04 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=1000.834, Time=0.08 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=1001.543, Time=0.09 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=998.718, Time=0.13 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=998.270, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=998.689, Time=0.06 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=998.059, Time=0.33 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=1003.480, Time=0.13 sec
 ARIMA(3,1,0)(0,0,0)[1] in

ARIMA(order=(1, 1, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,1,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(1, 1, 0)   Log Likelihood                -495.123
Method:                       css-mle   S.D. of innovations          42455.775
Date:                Sat, 11 Sep 2021   AIC                            996.246
Time:                        04:41:18   BIC                           1001.387
Sample:                             1   HQIC                           998.118
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        695.3995   5200.300      0.134      0.894   -9497.001    1.09e+04
ar.L1.D.y     -0.2821      0.148     -1.909      0.056      -0.572       0.008
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([146591.42208974, 146219.69947923]), array([42455.77501066, 52262.70880996]), array([[ 63379.63213312, 229803.21204637],
       [ 43786.67247719, 248652.72648126]]))
